Задание

Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
и задавались следующим образом:

x - последовательность цифр

y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10

Задача:
научить модель предсказывать y(i) по x(i)
пробовать RNN, LSTM, GRU

In [ ]:
import torch
import re
import random
import time
import numpy as np

In [ ]:
count_numbers = 5000
numbers_1 = 25
numbers_2 = 75
numbers_3 = 125

In [ ]:
x_1 = [[random.randint(0,9) for x in range(numbers_1)] for i in range(count_numbers)]
x_2 = [[random.randint(0,9) for x in range(numbers_2)] for i in range(count_numbers)]
x_3 = [[random.randint(0,9) for x in range(numbers_3)] for i in range(count_numbers)]

In [ ]:
y_1 = [[x_1[j][0]] + [x_1[j][0] + x_1[j][i] - 10 if x_1[j][0] + x_1[j][i] >= 10 else x_1[j][0] + x_1[j][i] for i in range(1, len(x_1[j]))] for j in range(len(x_1))]
y_2 = [[x_2[j][0]] + [x_2[j][0] + x_2[j][i] - 10 if x_2[j][0] + x_2[j][i] >= 10 else x_2[j][0] + x_2[j][i] for i in range(1, len(x_2[j]))] for j in range(len(x_2))]
y_3 = [[x_3[j][0]] + [x_3[j][0] + x_3[j][i] - 10 if x_3[j][0] + x_3[j][i] >= 10 else x_3[j][0] + x_3[j][i] for i in range(1, len(x_3[j]))] for j in range(len(x_3))]

Для проверки будем пересоздавать x и y заного рандомно и превращать в тензор

In [ ]:
X_1 = torch.tensor(x_1)
Y_1 = torch.tensor(y_1)
X_2 = torch.tensor(x_2)
Y_2 = torch.tensor(y_2)
X_3 = torch.tensor(x_3)
Y_3 = torch.tensor(y_3)

In [ ]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, rnnClass, num_len_size, embedding_size, num_hiddens):
        super().__init__()

        self.num_hiddens = num_hiddens
        self.embedding = torch.nn.Embedding(num_len_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = torch.nn.Linear(num_hiddens, num_len_size)

    def forward(self, X):
        out = self.embedding(X)
        out = self.hidden(out)[0]
        predictions = self.output(out).squeeze()
        return predictions

In [ ]:
def train_loop(model, optimizer, criterion, X, Y):
  for ep in range(300):
      start = time.time()
      train_loss = 0.
      train_passed = 0

      model.train()
      for X_b, y_b in zip(X, Y):
          optimizer.zero_grad()
          answers = model.forward(X_b)
          loss = criterion(answers, y_b)
          train_loss += loss.item()

          loss.backward()
          optimizer.step()
          train_passed += 1
          break
      if ep % 10 == 0:
        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

## RNN
Для проверки будем пересоздавать x и y заного рандомно и превращать в тензор

In [ ]:
model_1 = NeuralNetwork(torch.nn.RNN, 10, 64, 128)
model_2 = NeuralNetwork(torch.nn.RNN, 10, 64, 128)
model_3 = NeuralNetwork(torch.nn.RNN, 10, 64, 128)

In [ ]:
criterion_1 = torch.nn.CrossEntropyLoss()
optimizer_1 = torch.optim.Adam(model_1.parameters(), lr = 1e-3)
optimizer_2 = torch.optim.Adam(model_2.parameters(), lr = 1e-3)
optimizer_3 = torch.optim.Adam(model_3.parameters(), lr = 1e-3)

In [ ]:
train_loop(model_1, optimizer_1, criterion_1, X_1, Y_1)

Epoch 0. Time: 0.181, Train loss: 2.315
Epoch 10. Time: 0.116, Train loss: 0.932
Epoch 20. Time: 0.013, Train loss: 0.351
Epoch 30. Time: 0.013, Train loss: 0.151
Epoch 40. Time: 0.013, Train loss: 0.076
Epoch 50. Time: 0.019, Train loss: 0.044
Epoch 60. Time: 0.014, Train loss: 0.030
Epoch 70. Time: 0.014, Train loss: 0.022
Epoch 80. Time: 0.024, Train loss: 0.018
Epoch 90. Time: 0.023, Train loss: 0.015
Epoch 100. Time: 0.022, Train loss: 0.013
Epoch 110. Time: 0.026, Train loss: 0.011
Epoch 120. Time: 0.024, Train loss: 0.010
Epoch 130. Time: 0.022, Train loss: 0.008
Epoch 140. Time: 0.024, Train loss: 0.008
Epoch 150. Time: 0.018, Train loss: 0.007
Epoch 160. Time: 0.015, Train loss: 0.006
Epoch 170. Time: 0.013, Train loss: 0.006
Epoch 180. Time: 0.015, Train loss: 0.005
Epoch 190. Time: 0.013, Train loss: 0.005
Epoch 200. Time: 0.013, Train loss: 0.004
Epoch 210. Time: 0.013, Train loss: 0.004
Epoch 220. Time: 0.013, Train loss: 0.004
Epoch 230. Time: 0.013, Train loss: 0.004
Epo

In [ ]:
model_1(X_1[0]).argmax(dim=-1)

tensor([4, 0, 2, 3, 2, 4, 8, 9, 2, 8, 5, 0, 9, 0, 1, 0, 5, 6, 2, 6, 1, 5, 1, 0,
        9])

In [ ]:
Y_1[0]

tensor([4, 0, 2, 3, 2, 4, 8, 9, 2, 8, 5, 0, 9, 0, 1, 0, 5, 6, 2, 6, 1, 5, 1, 0,
        9])

In [ ]:
train_loop(model_2, optimizer_2, criterion_1, X_2, Y_2)

Epoch 0. Time: 0.027, Train loss: 2.453
Epoch 10. Time: 0.017, Train loss: 1.144
Epoch 20. Time: 0.023, Train loss: 0.465
Epoch 30. Time: 0.016, Train loss: 0.199
Epoch 40. Time: 0.015, Train loss: 0.102
Epoch 50. Time: 0.021, Train loss: 0.061
Epoch 60. Time: 0.016, Train loss: 0.041
Epoch 70. Time: 0.016, Train loss: 0.030
Epoch 80. Time: 0.015, Train loss: 0.024
Epoch 90. Time: 0.016, Train loss: 0.019
Epoch 100. Time: 0.016, Train loss: 0.016
Epoch 110. Time: 0.020, Train loss: 0.014
Epoch 120. Time: 0.015, Train loss: 0.012
Epoch 130. Time: 0.015, Train loss: 0.011
Epoch 140. Time: 0.016, Train loss: 0.009
Epoch 150. Time: 0.015, Train loss: 0.008
Epoch 160. Time: 0.017, Train loss: 0.008
Epoch 170. Time: 0.017, Train loss: 0.007
Epoch 180. Time: 0.017, Train loss: 0.006
Epoch 190. Time: 0.016, Train loss: 0.006
Epoch 200. Time: 0.016, Train loss: 0.005
Epoch 210. Time: 0.017, Train loss: 0.005
Epoch 220. Time: 0.016, Train loss: 0.005
Epoch 230. Time: 0.016, Train loss: 0.004
Epo

In [ ]:
Y_2[0]

tensor([5, 8, 8, 9, 7, 2, 8, 7, 2, 5, 7, 2, 3, 9, 9, 9, 1, 5, 1, 2, 7, 7, 7, 9,
        6, 3, 1, 9, 1, 4, 1, 7, 9, 5, 1, 8, 4, 4, 7, 8, 6, 5, 1, 3, 3, 1, 1, 6,
        5, 1, 8, 9, 9, 5, 4, 8, 2, 5, 2, 4, 9, 7, 1, 4, 6, 0, 9, 3, 7, 7, 8, 9,
        6, 6, 5])

In [ ]:
model_2(X_2[0]).argmax(dim=-1)

tensor([5, 8, 8, 9, 7, 2, 8, 7, 2, 5, 7, 2, 3, 9, 9, 9, 1, 5, 1, 2, 7, 7, 7, 9,
        6, 3, 1, 9, 1, 4, 1, 7, 9, 5, 1, 8, 4, 4, 7, 8, 6, 5, 1, 3, 3, 1, 1, 6,
        5, 1, 8, 9, 9, 5, 4, 8, 2, 5, 2, 4, 9, 7, 1, 4, 6, 0, 9, 3, 7, 7, 8, 9,
        6, 6, 5])

In [ ]:
train_loop(model_3, optimizer_3, criterion_1, X_3, Y_3)

Epoch 0. Time: 0.036, Train loss: 2.361
Epoch 10. Time: 0.038, Train loss: 1.127
Epoch 20. Time: 0.035, Train loss: 0.465
Epoch 30. Time: 0.020, Train loss: 0.202
Epoch 40. Time: 0.025, Train loss: 0.111
Epoch 50. Time: 0.019, Train loss: 0.074
Epoch 60. Time: 0.019, Train loss: 0.056
Epoch 70. Time: 0.019, Train loss: 0.045
Epoch 80. Time: 0.018, Train loss: 0.036
Epoch 90. Time: 0.019, Train loss: 0.029
Epoch 100. Time: 0.019, Train loss: 0.024
Epoch 110. Time: 0.020, Train loss: 0.020
Epoch 120. Time: 0.023, Train loss: 0.017
Epoch 130. Time: 0.018, Train loss: 0.014
Epoch 140. Time: 0.022, Train loss: 0.012
Epoch 150. Time: 0.019, Train loss: 0.011
Epoch 160. Time: 0.020, Train loss: 0.009
Epoch 170. Time: 0.019, Train loss: 0.008
Epoch 180. Time: 0.019, Train loss: 0.007
Epoch 190. Time: 0.020, Train loss: 0.007
Epoch 200. Time: 0.023, Train loss: 0.006
Epoch 210. Time: 0.025, Train loss: 0.006
Epoch 220. Time: 0.022, Train loss: 0.005
Epoch 230. Time: 0.021, Train loss: 0.005
Epo

In [ ]:
Y_3[0]

tensor([8, 2, 7, 6, 8, 2, 2, 5, 5, 6, 2, 3, 7, 6, 6, 2, 0, 0, 0, 8, 4, 6, 6, 0,
        8, 3, 9, 9, 8, 9, 6, 3, 4, 5, 0, 0, 1, 5, 2, 7, 0, 0, 5, 9, 3, 8, 8, 0,
        4, 4, 7, 8, 1, 0, 6, 2, 3, 0, 7, 6, 2, 1, 8, 9, 9, 2, 2, 4, 9, 3, 5, 9,
        3, 4, 1, 5, 8, 7, 6, 2, 1, 3, 7, 3, 6, 8, 3, 5, 1, 0, 9, 6, 7, 5, 9, 5,
        0, 4, 5, 9, 9, 2, 2, 0, 3, 7, 1, 8, 0, 8, 5, 6, 8, 8, 5, 9, 1, 2, 3, 3,
        3, 0, 1, 4, 5])

In [ ]:
model_3(X_3[0]).argmax(dim=-1)

tensor([8, 2, 7, 6, 8, 2, 2, 5, 5, 6, 2, 3, 7, 6, 6, 2, 0, 0, 0, 8, 4, 6, 6, 0,
        8, 3, 9, 9, 8, 9, 6, 3, 4, 5, 0, 0, 1, 5, 2, 7, 0, 0, 5, 9, 3, 8, 8, 0,
        4, 4, 7, 8, 1, 0, 6, 2, 3, 0, 7, 6, 2, 1, 8, 9, 9, 2, 2, 4, 9, 3, 5, 9,
        3, 4, 1, 5, 8, 7, 6, 2, 1, 3, 7, 3, 6, 8, 3, 5, 1, 0, 9, 6, 7, 5, 9, 5,
        0, 4, 5, 9, 9, 2, 2, 0, 3, 7, 1, 8, 0, 8, 5, 6, 8, 8, 5, 9, 1, 2, 3, 3,
        3, 0, 1, 4, 5])

## GRU
Для проверки будем пересоздавать x и y заного рандомно и превращать в тензор

In [ ]:
model_1_g = NeuralNetwork(torch.nn.GRU, 10, 64, 128)
model_2_g = NeuralNetwork(torch.nn.GRU, 10, 64, 128)
model_3_g = NeuralNetwork(torch.nn.GRU, 10, 64, 128)

In [ ]:
criterion_2 = torch.nn.CrossEntropyLoss()
optimizer_1 = torch.optim.Adam(model_1_g.parameters(), lr = 1e-3)
optimizer_2 = torch.optim.Adam(model_2_g.parameters(), lr = 1e-3)
optimizer_3 = torch.optim.Adam(model_3_g.parameters(), lr = 1e-3)

In [ ]:
train_loop(model_1_g, optimizer_1, criterion_2, X_1, Y_1)

Epoch 0. Time: 0.023, Train loss: 2.412
Epoch 10. Time: 0.018, Train loss: 1.478
Epoch 20. Time: 0.130, Train loss: 0.761
Epoch 30. Time: 0.020, Train loss: 0.326
Epoch 40. Time: 0.017, Train loss: 0.148
Epoch 50. Time: 0.017, Train loss: 0.076
Epoch 60. Time: 0.016, Train loss: 0.042
Epoch 70. Time: 0.018, Train loss: 0.026
Epoch 80. Time: 0.028, Train loss: 0.018
Epoch 90. Time: 0.153, Train loss: 0.014
Epoch 100. Time: 0.027, Train loss: 0.012
Epoch 110. Time: 0.029, Train loss: 0.010
Epoch 120. Time: 0.029, Train loss: 0.008
Epoch 130. Time: 0.028, Train loss: 0.007
Epoch 140. Time: 0.131, Train loss: 0.006
Epoch 150. Time: 0.016, Train loss: 0.006
Epoch 160. Time: 0.016, Train loss: 0.005
Epoch 170. Time: 0.017, Train loss: 0.005
Epoch 180. Time: 0.016, Train loss: 0.004
Epoch 190. Time: 0.126, Train loss: 0.004
Epoch 200. Time: 0.015, Train loss: 0.004
Epoch 210. Time: 0.016, Train loss: 0.003
Epoch 220. Time: 0.018, Train loss: 0.003
Epoch 230. Time: 0.017, Train loss: 0.003
Epo

In [ ]:
Y_1[0]

tensor([4, 0, 2, 3, 2, 4, 8, 9, 2, 8, 5, 0, 9, 0, 1, 0, 5, 6, 2, 6, 1, 5, 1, 0,
        9])

In [ ]:
model_1_g(X_1[0]).argmax(dim=-1)

tensor([4, 0, 2, 3, 2, 4, 8, 9, 2, 8, 5, 0, 9, 0, 1, 0, 5, 6, 2, 6, 1, 5, 1, 0,
        9])

In [ ]:
train_loop(model_2_g, optimizer_2, criterion_2, X_2, Y_2)

Epoch 0. Time: 0.029, Train loss: 2.337
Epoch 10. Time: 0.026, Train loss: 1.494
Epoch 20. Time: 0.026, Train loss: 0.791
Epoch 30. Time: 0.025, Train loss: 0.325
Epoch 40. Time: 0.132, Train loss: 0.138
Epoch 50. Time: 0.026, Train loss: 0.073
Epoch 60. Time: 0.027, Train loss: 0.046
Epoch 70. Time: 0.027, Train loss: 0.031
Epoch 80. Time: 0.025, Train loss: 0.022
Epoch 90. Time: 0.126, Train loss: 0.017
Epoch 100. Time: 0.025, Train loss: 0.014
Epoch 110. Time: 0.026, Train loss: 0.011
Epoch 120. Time: 0.032, Train loss: 0.010
Epoch 130. Time: 0.027, Train loss: 0.008
Epoch 140. Time: 0.125, Train loss: 0.007
Epoch 150. Time: 0.030, Train loss: 0.007
Epoch 160. Time: 0.033, Train loss: 0.006
Epoch 170. Time: 0.046, Train loss: 0.005
Epoch 180. Time: 0.043, Train loss: 0.005
Epoch 190. Time: 0.168, Train loss: 0.004
Epoch 200. Time: 0.042, Train loss: 0.004
Epoch 210. Time: 0.025, Train loss: 0.004
Epoch 220. Time: 0.028, Train loss: 0.003
Epoch 230. Time: 0.026, Train loss: 0.003
Epo

In [ ]:
Y_2[0]

tensor([5, 8, 8, 9, 7, 2, 8, 7, 2, 5, 7, 2, 3, 9, 9, 9, 1, 5, 1, 2, 7, 7, 7, 9,
        6, 3, 1, 9, 1, 4, 1, 7, 9, 5, 1, 8, 4, 4, 7, 8, 6, 5, 1, 3, 3, 1, 1, 6,
        5, 1, 8, 9, 9, 5, 4, 8, 2, 5, 2, 4, 9, 7, 1, 4, 6, 0, 9, 3, 7, 7, 8, 9,
        6, 6, 5])

In [ ]:
model_2_g(X_2[0]).argmax(dim=-1)

tensor([5, 8, 8, 9, 7, 2, 8, 7, 2, 5, 7, 2, 3, 9, 9, 9, 1, 5, 1, 2, 7, 7, 7, 9,
        6, 3, 1, 9, 1, 4, 1, 7, 9, 5, 1, 8, 4, 4, 7, 8, 6, 5, 1, 3, 3, 1, 1, 6,
        5, 1, 8, 9, 9, 5, 4, 8, 2, 5, 2, 4, 9, 7, 1, 4, 6, 0, 9, 3, 7, 7, 8, 9,
        6, 6, 5])

In [ ]:
train_loop(model_3_g, optimizer_3, criterion_2, X_3, Y_3)

Epoch 0. Time: 0.039, Train loss: 2.348
Epoch 10. Time: 0.053, Train loss: 1.564
Epoch 20. Time: 0.039, Train loss: 0.889
Epoch 30. Time: 0.041, Train loss: 0.393
Epoch 40. Time: 0.143, Train loss: 0.158
Epoch 50. Time: 0.037, Train loss: 0.081
Epoch 60. Time: 0.035, Train loss: 0.053
Epoch 70. Time: 0.036, Train loss: 0.040
Epoch 80. Time: 0.037, Train loss: 0.031
Epoch 90. Time: 0.137, Train loss: 0.024
Epoch 100. Time: 0.037, Train loss: 0.019
Epoch 110. Time: 0.039, Train loss: 0.015
Epoch 120. Time: 0.039, Train loss: 0.013
Epoch 130. Time: 0.037, Train loss: 0.010
Epoch 140. Time: 0.205, Train loss: 0.009
Epoch 150. Time: 0.060, Train loss: 0.008
Epoch 160. Time: 0.059, Train loss: 0.007
Epoch 170. Time: 0.036, Train loss: 0.006
Epoch 180. Time: 0.035, Train loss: 0.005
Epoch 190. Time: 0.140, Train loss: 0.005
Epoch 200. Time: 0.035, Train loss: 0.004
Epoch 210. Time: 0.037, Train loss: 0.004
Epoch 220. Time: 0.042, Train loss: 0.004
Epoch 230. Time: 0.036, Train loss: 0.003
Epo

In [ ]:
Y_3[0]

tensor([8, 2, 7, 6, 8, 2, 2, 5, 5, 6, 2, 3, 7, 6, 6, 2, 0, 0, 0, 8, 4, 6, 6, 0,
        8, 3, 9, 9, 8, 9, 6, 3, 4, 5, 0, 0, 1, 5, 2, 7, 0, 0, 5, 9, 3, 8, 8, 0,
        4, 4, 7, 8, 1, 0, 6, 2, 3, 0, 7, 6, 2, 1, 8, 9, 9, 2, 2, 4, 9, 3, 5, 9,
        3, 4, 1, 5, 8, 7, 6, 2, 1, 3, 7, 3, 6, 8, 3, 5, 1, 0, 9, 6, 7, 5, 9, 5,
        0, 4, 5, 9, 9, 2, 2, 0, 3, 7, 1, 8, 0, 8, 5, 6, 8, 8, 5, 9, 1, 2, 3, 3,
        3, 0, 1, 4, 5])

In [ ]:
model_3_g(X_3[0]).argmax(dim=-1)

tensor([8, 2, 7, 6, 8, 2, 2, 5, 5, 6, 2, 3, 7, 6, 6, 2, 0, 0, 0, 8, 4, 6, 6, 0,
        8, 3, 9, 9, 8, 9, 6, 3, 4, 5, 0, 0, 1, 5, 2, 7, 0, 0, 5, 9, 3, 8, 8, 0,
        4, 4, 7, 8, 1, 0, 6, 2, 3, 0, 7, 6, 2, 1, 8, 9, 9, 2, 2, 4, 9, 3, 5, 9,
        3, 4, 1, 5, 8, 7, 6, 2, 1, 3, 7, 3, 6, 8, 3, 5, 1, 0, 9, 6, 7, 5, 9, 5,
        0, 4, 5, 9, 9, 2, 2, 0, 3, 7, 1, 8, 0, 8, 5, 6, 8, 8, 5, 9, 1, 2, 3, 3,
        3, 0, 1, 4, 5])

## LSTM
Для проверки будем пересоздавать x и y заного рандомно и превращать в тензор

In [ ]:
model_1_l = NeuralNetwork(torch.nn.LSTM, 10, 64, 128)
model_2_l = NeuralNetwork(torch.nn.LSTM, 10, 64, 128)
model_3_l = NeuralNetwork(torch.nn.LSTM, 10, 64, 128)

In [ ]:
criterion_3 = torch.nn.CrossEntropyLoss()
optimizer_1 = torch.optim.Adam(model_1_l.parameters(), lr = 1e-3)
optimizer_2 = torch.optim.Adam(model_2_l.parameters(), lr = 1e-3)
optimizer_3 = torch.optim.Adam(model_3_l.parameters(), lr = 1e-3)

In [ ]:
train_loop(model_1_l, optimizer_1, criterion_3, X_1, Y_1)

Epoch 0. Time: 0.098, Train loss: 2.307
Epoch 10. Time: 0.013, Train loss: 1.756
Epoch 20. Time: 0.013, Train loss: 1.099
Epoch 30. Time: 0.014, Train loss: 0.553
Epoch 40. Time: 0.016, Train loss: 0.265
Epoch 50. Time: 0.013, Train loss: 0.143
Epoch 60. Time: 0.013, Train loss: 0.085
Epoch 70. Time: 0.013, Train loss: 0.053
Epoch 80. Time: 0.013, Train loss: 0.035
Epoch 90. Time: 0.015, Train loss: 0.025
Epoch 100. Time: 0.012, Train loss: 0.020
Epoch 110. Time: 0.013, Train loss: 0.016
Epoch 120. Time: 0.013, Train loss: 0.013
Epoch 130. Time: 0.014, Train loss: 0.011
Epoch 140. Time: 0.015, Train loss: 0.010
Epoch 150. Time: 0.012, Train loss: 0.009
Epoch 160. Time: 0.013, Train loss: 0.008
Epoch 170. Time: 0.014, Train loss: 0.007
Epoch 180. Time: 0.022, Train loss: 0.006
Epoch 190. Time: 0.025, Train loss: 0.006
Epoch 200. Time: 0.022, Train loss: 0.005
Epoch 210. Time: 0.028, Train loss: 0.005
Epoch 220. Time: 0.023, Train loss: 0.004
Epoch 230. Time: 0.022, Train loss: 0.004
Epo

In [ ]:
Y_1[0]

tensor([4, 0, 2, 3, 2, 4, 8, 9, 2, 8, 5, 0, 9, 0, 1, 0, 5, 6, 2, 6, 1, 5, 1, 0,
        9])

In [ ]:
model_1_l(X_1[0]).argmax(dim=-1)

tensor([4, 0, 2, 3, 2, 4, 8, 9, 2, 8, 5, 0, 9, 0, 1, 0, 5, 6, 2, 6, 1, 5, 1, 0,
        9])

In [ ]:
train_loop(model_2_l, optimizer_2, criterion_3, X_2, Y_2)

Epoch 0. Time: 0.021, Train loss: 2.308
Epoch 10. Time: 0.014, Train loss: 1.805
Epoch 20. Time: 0.014, Train loss: 1.212
Epoch 30. Time: 0.017, Train loss: 0.643
Epoch 40. Time: 0.015, Train loss: 0.296
Epoch 50. Time: 0.014, Train loss: 0.149
Epoch 60. Time: 0.014, Train loss: 0.089
Epoch 70. Time: 0.013, Train loss: 0.061
Epoch 80. Time: 0.013, Train loss: 0.045
Epoch 90. Time: 0.016, Train loss: 0.035
Epoch 100. Time: 0.013, Train loss: 0.027
Epoch 110. Time: 0.014, Train loss: 0.022
Epoch 120. Time: 0.022, Train loss: 0.018
Epoch 130. Time: 0.015, Train loss: 0.015
Epoch 140. Time: 0.018, Train loss: 0.013
Epoch 150. Time: 0.013, Train loss: 0.011
Epoch 160. Time: 0.015, Train loss: 0.010
Epoch 170. Time: 0.015, Train loss: 0.009
Epoch 180. Time: 0.013, Train loss: 0.008
Epoch 190. Time: 0.015, Train loss: 0.007
Epoch 200. Time: 0.013, Train loss: 0.006
Epoch 210. Time: 0.015, Train loss: 0.006
Epoch 220. Time: 0.013, Train loss: 0.005
Epoch 230. Time: 0.013, Train loss: 0.005
Epo

In [ ]:
Y_2[0]

tensor([5, 8, 8, 9, 7, 2, 8, 7, 2, 5, 7, 2, 3, 9, 9, 9, 1, 5, 1, 2, 7, 7, 7, 9,
        6, 3, 1, 9, 1, 4, 1, 7, 9, 5, 1, 8, 4, 4, 7, 8, 6, 5, 1, 3, 3, 1, 1, 6,
        5, 1, 8, 9, 9, 5, 4, 8, 2, 5, 2, 4, 9, 7, 1, 4, 6, 0, 9, 3, 7, 7, 8, 9,
        6, 6, 5])

In [ ]:
model_2_l(X_2[0]).argmax(dim=-1)

tensor([5, 8, 8, 9, 7, 2, 8, 7, 2, 5, 7, 2, 3, 9, 9, 9, 1, 5, 1, 2, 7, 7, 7, 9,
        6, 3, 1, 9, 1, 4, 1, 7, 9, 5, 1, 8, 4, 4, 7, 8, 6, 5, 1, 3, 3, 1, 1, 6,
        5, 1, 8, 9, 9, 5, 4, 8, 2, 5, 2, 4, 9, 7, 1, 4, 6, 0, 9, 3, 7, 7, 8, 9,
        6, 6, 5])

In [ ]:
train_loop(model_3_l, optimizer_3, criterion_3, X_3, Y_3)

Epoch 0. Time: 0.028, Train loss: 2.309
Epoch 10. Time: 0.014, Train loss: 1.837
Epoch 20. Time: 0.015, Train loss: 1.250
Epoch 30. Time: 0.014, Train loss: 0.646
Epoch 40. Time: 0.026, Train loss: 0.274
Epoch 50. Time: 0.014, Train loss: 0.130
Epoch 60. Time: 0.016, Train loss: 0.078
Epoch 70. Time: 0.015, Train loss: 0.056
Epoch 80. Time: 0.027, Train loss: 0.043
Epoch 90. Time: 0.031, Train loss: 0.035
Epoch 100. Time: 0.026, Train loss: 0.028
Epoch 110. Time: 0.026, Train loss: 0.024
Epoch 120. Time: 0.027, Train loss: 0.020
Epoch 130. Time: 0.026, Train loss: 0.017
Epoch 140. Time: 0.015, Train loss: 0.014
Epoch 150. Time: 0.014, Train loss: 0.013
Epoch 160. Time: 0.014, Train loss: 0.011
Epoch 170. Time: 0.015, Train loss: 0.010
Epoch 180. Time: 0.016, Train loss: 0.009
Epoch 190. Time: 0.016, Train loss: 0.008
Epoch 200. Time: 0.015, Train loss: 0.007
Epoch 210. Time: 0.015, Train loss: 0.006
Epoch 220. Time: 0.020, Train loss: 0.006
Epoch 230. Time: 0.015, Train loss: 0.005
Epo

In [ ]:
Y_3[0]

tensor([8, 2, 7, 6, 8, 2, 2, 5, 5, 6, 2, 3, 7, 6, 6, 2, 0, 0, 0, 8, 4, 6, 6, 0,
        8, 3, 9, 9, 8, 9, 6, 3, 4, 5, 0, 0, 1, 5, 2, 7, 0, 0, 5, 9, 3, 8, 8, 0,
        4, 4, 7, 8, 1, 0, 6, 2, 3, 0, 7, 6, 2, 1, 8, 9, 9, 2, 2, 4, 9, 3, 5, 9,
        3, 4, 1, 5, 8, 7, 6, 2, 1, 3, 7, 3, 6, 8, 3, 5, 1, 0, 9, 6, 7, 5, 9, 5,
        0, 4, 5, 9, 9, 2, 2, 0, 3, 7, 1, 8, 0, 8, 5, 6, 8, 8, 5, 9, 1, 2, 3, 3,
        3, 0, 1, 4, 5])

In [ ]:
model_3_l(X_3[0]).argmax(dim=-1)

tensor([8, 2, 7, 6, 8, 2, 2, 5, 5, 6, 2, 3, 7, 6, 6, 2, 0, 0, 0, 8, 4, 6, 6, 0,
        8, 3, 9, 9, 8, 9, 6, 3, 4, 5, 0, 0, 1, 5, 2, 7, 0, 0, 5, 9, 3, 8, 8, 0,
        4, 4, 7, 8, 1, 0, 6, 2, 3, 0, 7, 6, 2, 1, 8, 9, 9, 2, 2, 4, 9, 3, 5, 9,
        3, 4, 1, 5, 8, 7, 6, 2, 1, 3, 7, 3, 6, 8, 3, 5, 1, 0, 9, 6, 7, 5, 9, 5,
        0, 4, 5, 9, 9, 2, 2, 0, 3, 7, 1, 8, 0, 8, 5, 6, 8, 8, 5, 9, 1, 2, 3, 3,
        3, 0, 1, 4, 5])